<a href="https://colab.research.google.com/github/varunsawhney8/Projects/blob/main/sample20Aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np


path='/content/Reliance_Final_Ndtv_data4.xlsx'
data=pd.read_excel(path)


data=data[::-1].reset_index()
data.drop(columns=['index'],axis=True,inplace=True)

path='/content/Reliance_Nse_data2.csv'
data1=pd.read_csv(path)


data['Date']=data['Date'].astype('str')

data["Price"]=[np.nan for i in range(data.shape[0])]




for i in range(len(data['Date'])):
    for j in range(len(data1['Date'])):
        if data['Date'][i]==data1['Date'][j]:
            data["Price"][i]=data1["Close"][j]
        

data["Price"]=data["Price"].fillna(method='bfill')

data["Price"][372:]=2174



# =============================================================================
# # Simple Moving Average
# 
# data1['SMA_3'] = data1.iloc[:,1].rolling(window=3).mean().round()
# 
# # Weighted Moving Average
# 
# weights = np.arange(1,11)
# wma10 = data1['Close'].rolling(10).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)
# 
# 
# =============================================================================
# Exponential Moving Average

#data['7day_ea'] = data['Price'].ewm(span=7).mean()

data.columns
data.drop(columns=['Media','Article','Keywords'],inplace=True,axis=1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
data.columns

Index(['Date', 'Title', 'Price'], dtype='object')

In [30]:
!pip install transformers
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model="ProsusAI/finbert")
list3=[]

for i in range(len(data['Title'])):
    a=(classifier(data['Title'][i]))
    a=a[0]
    a=list(a.values())[0]

    if a=='positive':
        a=1
        list3.append(a)
    elif a=='negative' :
        a=2
        list3.append(a)
    else:
        a=0
        list3.append(a)

data['finbert']=list3

del list3


In [31]:
data.head()

,Date,Title,Price,finbert
0,2020-07-16,Reliance's Stake Sale Talks With Saudi Aramco ...,1843.400024,2
1,2020-07-17,Reliance To Shut Crude Refining Unit For Maint...,1911.699951,2
2,2020-07-20,Reliance-Google Smartphone Deal in India Threa...,1919.900024,2
3,2020-07-21,"Netflix, Amazon, Facebook Stocks Draw In Amate...",1971.550049,1
4,2020-07-21,JioMart App for Android and iOS Launched: Here...,1971.550049,0


In [40]:
aggregation_functions = {'Price': 'mean', 'Title': 'first', 'finbert': 'mean'}
data = data.groupby(data['Date']).aggregate(aggregation_functions)

In [41]:
#data.head()
data['7day_sentiment'] = round(data['finbert'].ewm(span=7).mean())

In [42]:
data.columns

Index(['Price', 'Title', 'finbert', '7day_sentiment'], dtype='object')

In [43]:
data['perc_change'] =data['Price'].pct_change(periods=1)*100

In [45]:
data.head(175)

,Price,Title,finbert,7day_sentiment,perc_change
Date,,,,,
2020-07-16,1843.400024,Reliance's Stake Sale Talks With Saudi Aramco ...,2.000000,2.0,NaN
2020-07-17,1911.699951,Reliance To Shut Crude Refining Unit For Maint...,2.000000,2.0,3.705106
2020-07-20,1919.900024,Reliance-Google Smartphone Deal in India Threa...,2.000000,2.0,0.428941
2020-07-21,1971.550049,"Netflix, Amazon, Facebook Stocks Draw In Amate...",0.500000,1.0,2.690246
2020-07-22,2004.000000,Reliance Industries Postpones June Quarter Res...,1.000000,1.0,1.645911
...,...,...,...,...,...
2021-08-06,2089.050049,"In Huge Reliance vs Amazon Clash, Supreme Cour...",1.000000,1.0,-0.698750
2021-08-08,2088.050049,Bankers In A Huddle As Court Rulings Threaten ...,2.000000,1.0,-0.047869
2021-08-10,2088.050049,Nifty Seen Opening Marginally Lower; Reliance ...,1.000000,1.0,0.000000
